In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import os, sys
sys.path.append('..')
from glob import glob
from configs import *
from tqdm import tqdm
from utils import *
import numpy as np
import matplotlib.pyplot as plt
from dataset import *
from time import time
tqdm.monitor_interval = 0
import torch
import json
import torch.nn as nn
import numpy as np
from time import time, strptime, mktime
torch.manual_seed(0)
np.random.seed(0)

In [4]:
sample_file = '/home/ubuntu/datasets-aux/MTA-Bus-Time_.2014-10-30.txt'

In [5]:
def collect(byf, fname, verbose=True):
    res = []
    li = 0
    added = 0
    with open('/home/ubuntu/datasets-aux/MTA-Bus-Time_.2014-10-30.txt') as fl:
        _ = fl.readline()
        line = fl.readline()
        while line:
            if 'NULL' not in line and 'IN_PROGRESS' in line:
                obj = byf(line)
                if obj is not None:
                    res.append(obj)
                    added += 1
            line = fl.readline()
            li += 1
            if li % 10000 == 0 and verbose:
                sys.stdout.write('%d       \r' % li)
        if verbose: sys.stdout.flush()
    if verbose: print('Collected %d/%d' % (added, li))
    return res

def by(line):
#     if root in line:
    try:
        fobj = fmt(line)
        bus_id = fobj['routeid'].split('_')[1]
        if 'X' not in bus_id:
            return fobj
    except:
        print(line)
        assert False
    return None

raw_seghist = collect(by, sample_file)

Collected 3827194/5503372


In [6]:
print(len(raw_seghist))

3827194


In [228]:
byroute = group(raw_seghist, 'routeid')
print(len(byroute.keys()))

231


In [8]:
scodes = {}
for stop in raw_seghist:
    scode = stop['stop'].split('_')[1]
    scodes[scode] = True
print(len(scodes.keys()))
print(list(scodes.keys())[:5])

12719
['402644', '306665', '300529', '405155', '307819']


In [181]:
bybus = group(raw_seghist, 'busid')
print(len(bybus.keys()))

3582


In [126]:
raw_seghist[0]

{'busid': '3666',
 'direction': 0,
 'dist': 7784.000404746534,
 'phase': True,
 'routeid': 'MTABC_Q111',
 'stop': 'MTA_552345',
 'time': time.struct_time(tm_year=2014, tm_mon=10, tm_mday=30, tm_hour=4, tm_min=0, tm_sec=1, tm_wday=3, tm_yday=303, tm_isdst=-1)}

In [246]:
def next_stops(travel, adj={}):
    prevseg = None
    prev = travel[0]
    
    for _, stop in enumerate(travel[1:]):
        if stop['stop'] != prev['stop']:
            if np.abs(prev['dist'] - stop['dist']) > 2.5 * 1000:
                # sensor reset? or loop-around... either way unreliable
                # should be okay to ignore when inferring stops from lots of data
                pass
            elif stop['time'] - prev['time'] > 60 * 60:
                # skip if >1hr difference
                pass
            else:
                if prev['stop'] not in adj: adj[prev['stop']] = {}
                if stop['stop'] not in adj[prev['stop']]: adj[prev['stop']][stop['stop']] = []
                adj[prev['stop']][stop['stop']].append((prev, stop))
            prev = stop
#             pdist = stop['dist']
#     return adj

byroute = group(raw_seghist, 'routeid')
print(len(byroute.keys()))

route_adjs = {}
for rid, route in byroute.items():
# for rid, route in tqdm(byroute.items()):
    

    bydir = group(route, 'direction')
    for dir, direction in bydir.items():
        adj = {}
        bytravel = group(direction, 'busid', kf=lambda key: key.replace('+', ''))
        for bid, travel in bytravel.items():
            next_stops(travel, adj)

#         print(dir, rid, len(bytravel.keys()), len(adj))

        route_adjs['%d_%s' % (dir, rid)] = adj

231


In [247]:
print(len(route_adjs['1_MTA NYCT_M101']))
# route_adjs['1_MTA NYCT_M101']
for next, ls in route_adjs['1_MTA NYCT_M101']['MTA_903147'].items():
    print(next, len(ls), ls[0][1]['dist'], ls[0][0]['dist'])

80
MTA_903148 3 17080.51000699644 17098.10616502553


In [36]:
sname = '../data/next_stops/%s.json' % sample_file.split('.')[1]

with open(sname, 'w') as fl:
    json.dump(route_adjs, fl)

### Gathering data

In [217]:
dfiles = sorted(glob('../data/next_stops/*.json'))
print(len(dfiles))

90


In [218]:
all_adjs = {}
for fname in dfiles:
    with open(fname) as fl:
        data = json.load(fl)
    
#     print(len(data))
    for rid, byroute in data.items():
        if rid not in all_adjs: all_adjs[rid] = {}
        for sid, nexts in byroute.items():
            if sid not in all_adjs[rid]: all_adjs[rid][sid] = {}
            for next_stop, occur in nexts.items():
                if next_stop not in all_adjs[rid][sid]: all_adjs[rid][sid][next_stop] = 0
                all_adjs[rid][sid][next_stop] += occur
#     break

In [219]:
nMany = 0
nSingle = 0
occ_distrib = []
for rid, byroute in all_adjs.items():
    for stop, next_stops in byroute.items():
        if len(next_stops) > 1:
            nMany += 1
            for next, occur in next_stops.items():
                occ_distrib.append(occur)
        else:
            nSingle += 1
print(nMany, nSingle)
next_stops

27061 2100


{'MTA_306960': 3,
 'MTA_307997': 1,
 'MTA_503963': 1,
 'MTA_504409': 2,
 'MTA_504430': 1451,
 'MTA_504549': 7,
 'MTA_505215': 5288}

In [234]:
class Node:
    def __init__(self, seg):
        self.ch = []
        self.st = seg[0]
        self.ed = seg[1]
        self.occ = seg[2]

for rid, byroute in [('1_MTA NYCT_M101', all_adjs['1_MTA NYCT_M101'])]:
#     print(rid)
    segs = []
    for stop, next_stops in byroute.items():
        slist = list(next_stops.keys())
        slist = sorted(slist, key=lambda st: next_stops[st])
        smax = slist[-1]
        seg = (stop, smax, next_stops[smax])
        segs.append(seg)
    
    maxind = max(range(len(segs)), key=lambda ind: segs[ind][2])
    bag_of_segs = segs[::1]
    bag_of_segs.pop(maxind)
    seq = [segs[maxind]]
#     print(seq)
    found = True
    while found:
        found = False
        next_cands = []
        for si, (st, ed, occ) in enumerate(bag_of_segs):
            if st == seq[-1][1]:
                next_cands.append(si)
        if len(next_cands):
            next_best = max(next_cands, key=lambda ind: bag_of_segs[ind][2])
            seq.append(bag_of_segs[next_best])
            bag_of_segs.pop(next_best)
            found = True
            
        next_cands = [] # previous cands
        for si, (st, ed, occ) in enumerate(bag_of_segs):
            if ed == seq[0][0]:
                next_cands.append(si)
        if len(next_cands):
            next_best = max(next_cands, key=lambda ind: bag_of_segs[ind][2])
            seq = [bag_of_segs[next_best]] + seq
            bag_of_segs.pop(next_best)
            found = True
    
    print('bag', len(bag_of_segs))
#     for seg in bag_of_segs:
#         print(seg)
        
    print('seq', len(seq))
    for seg in seq:
        print(seg)
#     print(len(seq))
    break

bag 107
seq 50
('MTA_402769', 'MTA_405069', 8802)
('MTA_405069', 'MTA_402772', 8369)
('MTA_402772', 'MTA_402773', 7483)
('MTA_402773', 'MTA_402774', 6858)
('MTA_402774', 'MTA_405070', 7526)
('MTA_405070', 'MTA_404949', 8872)
('MTA_404949', 'MTA_402638', 9647)
('MTA_402638', 'MTA_402639', 9213)
('MTA_402639', 'MTA_402641', 9724)
('MTA_402641', 'MTA_402642', 9451)
('MTA_402642', 'MTA_402643', 8264)
('MTA_402643', 'MTA_402644', 8704)
('MTA_402644', 'MTA_402645', 9528)
('MTA_402645', 'MTA_402646', 9104)
('MTA_402646', 'MTA_402647', 8725)
('MTA_402647', 'MTA_405078', 8850)
('MTA_405078', 'MTA_401458', 9717)
('MTA_401458', 'MTA_401459', 7705)
('MTA_401459', 'MTA_402653', 7744)
('MTA_402653', 'MTA_402723', 9102)
('MTA_402723', 'MTA_402656', 7919)
('MTA_402656', 'MTA_402657', 8838)
('MTA_402657', 'MTA_402658', 9909)
('MTA_402658', 'MTA_402659', 10100)
('MTA_402659', 'MTA_402660', 10447)
('MTA_402660', 'MTA_402661', 9507)
('MTA_402661', 'MTA_402662', 9022)
('MTA_402662', 'MTA_404255', 9725)
('M

In [223]:
with open('../data/next_stops/2014-10-30.json') as fl:
    sdata = json.load(fl)

In [232]:
# list(sdata['1_MTA NYCT_M101'].keys())